This notebook is intended to have all the code to reproduce the results in the paper.

In [ ]:
from scipy.integrate import quad
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import animation
from IPython.display import HTML
from psystem_homog_ps import fine_resolution, solve_psystem_homog
from clawpack import pyclaw
font = {'size'   : 15}
matplotlib.rc('font', **font)
lw = 1.5

# These physical values are used throughout the notebook and should not be modified later.
# They must also match the values set at the top of psystem_homog_ps.py.
pstar = 1.  # Ambient pressure
vstar = 1.  # Ambent specific volume
gamma = 1.4

# Introductory example

## Spatially Periodic entropy

The data for this figure was generated by running

` mpirun -np 6 python3 Euler_Eulerian.py mx=120000 tfinal=500 nout=200 use_petsc=True outdir='./_Euler_120k_ng_sc' solver_type=sharpclaw`

In [ ]:
path = "./_Euler_120k_ng_sc/"

fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 1.1
props = dict(facecolor='lightgrey', alpha=0.2)

def load_frame(i):
    eul_sol = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path,read_aux=False)
    rho = eul_sol.state.q[0,:]
    rhou = eul_sol.state.q[1,:]
    E = eul_sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    xc = eul_sol.state.grid.x.centers
    t = eul_sol.t
    return P, xc, t

def add_plot(frame,axis,xmin,xmax):
    i = frame
    P, xc, t = load_frame(i)
    axis.plot(xc,P,'-k',lw=lw,alpha=1.,label='Euler')
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([1, 1.05, 1.1])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/10,ytext,'$t=%3.1f$' % (t),bbox=props)

add_plot(20,axis=ax[0,0],xmin=40,xmax=80)
#fig.legend(loc='upper center', fancybox=True, shadow=True);
add_plot(50,axis=ax[0,1],xmin=125,xmax=165)
add_plot(100,axis=ax[1,0],xmin=275,xmax=315)
add_plot(180,axis=ax[1,1],xmin=520,xmax=560)

fig.text(0.5, 0.04, '$\chi$', ha='center', va='center')
fig.text(0.06, 0.5, 'Pressure ($p$)', ha='center', va='center', rotation='vertical')
plt.savefig('intro_example.pdf')

## Spatilally Constant entropy

The data for this figure was generated by running

`python3 Euler_Eulerian.py variation="none" mx=12000 tfinal=500 nout=200 use_petsc=True outdir='./_Euler_const_rho' solver_type=sharpclaw`

In [ ]:
path = "./_Euler_const_rho/"

fig, ax = plt.subplots(1,2,figsize=(12,4),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 1.1
props = dict(facecolor='lightgrey', alpha=0.2)

def load_frame(i):
    eul_sol = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path,read_aux=False)
    rho = eul_sol.state.q[0,:]
    rhou = eul_sol.state.q[1,:]
    E = eul_sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    xc = eul_sol.state.grid.x.centers
    t = eul_sol.t
    return P, xc, t

def add_plot(frame,axis,xmin,xmax):
    i = frame
    P, xc, t = load_frame(i)
    axis.plot(xc,P,'-k',lw=lw,alpha=1.,label='Euler')
    axis.set_xlim(xmin,xmax); axis.set_ylim(0.995,1.12)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([1, 1.05, 1.1])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/10,ytext,'$t=%3.1f$' % (t),bbox=props)

add_plot(20,axis=ax[0],xmin=40,xmax=100)
#fig.legend(loc='upper center', fancybox=True, shadow=True);
add_plot(50,axis=ax[1],xmin=145,xmax=205)
#add_plot(100,axis=ax[1,0],xmin=275,xmax=315)
#add_plot(180,axis=ax[1,1],xmin=520,xmax=560)

fig.text(0.5, 0.04, '$\chi$', ha='center', va='center')
fig.text(0.06, 0.5, 'Pressure ($p$)', ha='center', va='center', rotation='vertical')
plt.savefig('intro_example_const_rho.pdf')

# Comparison of Euler, p-system, and homogenized solutions

In [ ]:
# Pseudospectral solution of the homogenized equations
# This takes a minute or two to run
SA = -gamma*np.log(1./4)
SB = -gamma*np.log(7./4)
tmax = 500
m = 4096
x_ps, xi_ps, p_ps, u_ps, _ = solve_psystem_homog(m=m,L=1200,p_amp=0.15,tmax=tmax,dtfac=0.5,make_anim=False,
                                        num_plots=200,order4=True,width=5.,SA=SA,SB=SB)

In [ ]:
path_eul = './_Euler_120k_ng_sc/'
path_lag = './_equal_mx320_L600/'

tmax = 500
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 1.1
props = dict(facecolor='lightgrey', alpha=0.2)
t_int = tmax/(len(p_ps)-1)

def load_frame(i):
    eul_sol = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path_eul,read_aux=False)
    rho = eul_sol.state.q[0,:]
    rhou = eul_sol.state.q[1,:]
    E = eul_sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    exic = np.cumsum(rho)*eul_sol.state.grid.delta[0]

    lag_sol = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path_lag,read_aux=True)
    v = lag_sol.state.q[0,:]
    s = lag_sol.state.aux[0,:]
    pp = pstar * np.exp(s) * (vstar/v)**gamma
    lxic = lag_sol.state.grid.x.centers

    return P, exic, pp, lxic

def add_plot(frame,axis,xmin,xmax):
    i = frame
    axis = axis
    xfine, pfine = fine_resolution(p_ps[i],10000,x_ps,xi_ps)
    #axis.set_ylim(0.99,1.17)
    P, exic, pp, lxic = load_frame(i)
    axis.plot(exic,P,'-k',lw=lw,alpha=1.,label='Euler')
    axis.plot(lxic,pp,'--',lw=lw,alpha=0.8,label='p-system')
    axis.plot(xfine,pfine,'-.r',lw=lw, label='Homogenized p-system')
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([1, 1.05, 1.1])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/10,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(20,axis=ax[0,0],xmin=40,xmax=80)
fig.legend(loc='upper center', fancybox=True, shadow=True);
add_plot(50,axis=ax[0,1],xmin=125,xmax=165)
add_plot(100,axis=ax[1,0],xmin=275,xmax=315)
add_plot(180,axis=ax[1,1],xmin=520,xmax=560)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, 'Pressure ($p$)', ha='center', va='center', rotation='vertical')
plt.savefig('comparison3.pdf')

# Large-amplitude (shock) example

In [ ]:
# Pseudospectral solution of the homogenized equations
# This takes a minute or two to run
SA = -gamma*np.log(1./4)
SB = -gamma*np.log(7./4)
tmax = 500
m = 4096
x_ps, xi_ps, p_ps, u_ps, _ = solve_psystem_homog(m=m,L=1200,p_amp=0.50,tmax=tmax,dtfac=0.5,make_anim=False,
                                        num_plots=200,order4=True,width=5.,SA=SA,SB=SB)

To generate the data for the next figure:

`python3 Euler_Eulerian.py mx=120000 tfinal=500 nout=200 use_petsc=True outdir='./_Euler_120k_ng_sc_A50' amp=0.5 solver_type=sharpclaw`

`mpirun -np 6 python3 psystem_solitary.py amp=0.5 cells_per_layer=160 outdir=_equal_mx160_L600_A50 use_petsc=1 nout=200 L=600`

In [ ]:
path_eul = "./_Euler_120k_ng_sc_A50/"
path_lag = "./_equal_mx160_L600_A50/"

tmax = 500
fig, ax = plt.subplots(2,2,figsize=(12,8),sharey=True)
plt.subplots_adjust(wspace=0.02)
ytext = 1.35
props = dict(facecolor='lightgrey', alpha=0.2)
t_int = tmax/(len(p_ps)-1)

def load_frame(i):
    eul_sol = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path_eul,read_aux=False)
    rho = eul_sol.state.q[0,:]
    rhou = eul_sol.state.q[1,:]
    E = eul_sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    exic = np.cumsum(rho)*eul_sol.state.grid.delta[0]

    lag_sol = pyclaw.Solution(i,file_format='petsc',file_prefix='claw',path=path_lag,read_aux=True)
    v = lag_sol.state.q[0,:]
    s = lag_sol.state.aux[0,:]
    pp = pstar * np.exp(s) * (vstar/v)**gamma
    lxic = lag_sol.state.grid.x.centers

    return P, exic, pp, lxic

def add_plot(frame,axis,xmin,xmax):
    i = frame
    axis = axis
    xfine, pfine = fine_resolution(p_ps[i],10000,x_ps,xi_ps)
#    P, exic = load_frame(i)
    P, exic, pp, lxic = load_frame(i)
    axis.plot(exic,P,'-k',lw=lw,alpha=1.,label='Euler')
    axis.plot(lxic,pp,'--',lw=lw,alpha=0.8,label='p-system')
    axis.plot(xfine,pfine,'-.r',lw=lw, label='Homogenized p-system')
    axis.set_xlim(xmin,xmax)
    axis.set_xticks(range(int(xmin)+5, int(xmax)-4, 10))
    axis.set_yticks([1, 1.2, 1.4])  # set only 3 y-axis tick labels
    axis.text(xmin+(xmax-xmin)/10,ytext,'$t=%3.1f$' % (i*t_int),bbox=props)

add_plot(10,axis=ax[0,0],xmin=10,xmax=50)
fig.legend(loc='upper center', fancybox=True, shadow=True);
add_plot(50,axis=ax[0,1],xmin=130,xmax=170)
add_plot(100,axis=ax[1,0],xmin=270,xmax=340)
add_plot(180,axis=ax[1,1],xmin=520,xmax=600)

fig.text(0.5, 0.04, 'x', ha='center', va='center')
fig.text(0.06, 0.5, 'Pressure ($p$)', ha='center', va='center', rotation='vertical')
plt.savefig('comparison3_shock.pdf')

# Comparison of FV and PS solutions with smoothly-varying (sinusoidal) entropy

The data from Clawpack for this plot is generated by running

`python3 Eulerian_IC.py mx=144000 nout=200 outdir="./_ps_fv_comparison_sc_144k_cos" use_petsc=0 tfinal=400 solver_type=sharpclaw`

In [ ]:
# Load pseudospectral data generated from MATLAB
import scipy.io
soln_ps = scipy.io.loadmat("/Users/ketch/Dropbox/Ketcheson/Euler_576k/data_t400.mat")
pressure_ps = soln_ps["U"][2,:]
x_ps = soln_ps["x"][0]

# Load Clawpack simulation data
i = 200
path = "./_ps_fv_comparison_sc_144k_cos" # SharpClaw
sol = pyclaw.Solution(i,path=path,read_aux=False)
xc = sol.state.grid.x.centers

rho = sol.state.q[0,:]
rhou = sol.state.q[1,:]
E = sol.state.q[2,:]
P = (gamma-1)*(E-0.5*rhou**2/rho)

fig, ax = plt.subplots(figsize=(10,4))

ax.plot(xc,P,"r",lw=0.75)
ax.plot(x_ps,pressure_ps,"--k",lw=0.75)
ax.set_xlim(-500,500)
ax.set_ylim(0.99,1.14)

x1, x2, y1, y2 = -20, 20, 0.998, 1.002  # subregion of the original image
axins = ax.inset_axes(
    [-400, 1.04, 400, 0.06],
    xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[], transform=ax.transData)
axins.plot(xc,P,"r")
axins.plot(x_ps,pressure_ps,"--k",lw=1)

ax.indicate_inset_zoom(axins, edgecolor="black");

x1, x2, y1, y2 = 460, 495, 0.995, 1.12  # subregion of the original image
axins2 = ax.inset_axes(
    [100, 1.03, 300, 0.1],
    xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[], transform=ax.transData)
axins2.plot(xc,P,"r")
axins2.plot(x_ps,pressure_ps,"--k",lw=1)

ax.indicate_inset_zoom(axins2, edgecolor="black");

ax.set_xlabel("x");
ax.set_ylabel("Pressure");

plt.savefig("ps_fv_comparison.pdf", bbox_inches='tight');

# Solitary wave comparison

The Clawpack data for the next figure is generated by running

`mpirun -np 6 python3 Euler_Eulerian.py mx=280000 xright=700 tfinal=600 nout=20 use_petsc=True outdir='./_Euler_280k_sc_gauges' solver_type=sharpclaw gauges=1`

In [ ]:
def get_pressure(gaugefile):
    gauge = np.loadtxt(gaugefile)
    tt = gauge[:,0]
    rho = gauge[:,1]
    rhou = gauge[:,2]
    E = gauge[:,3]
    rhoe = E - 0.5*rhou**2/rho
    gamma = 1.4
    p = (gamma-1)*rhoe
    return tt, p
    
plt.figure(figsize=(8,4))
v = 10./8.18
shift = -647.62
tt, p = get_pressure("./_Euler_280k_sc_gauges/_gauges/gauge650.0.txt")
plt.plot(v*tt+shift,p,"r",label="FV solution")

tt, p = get_pressure("./_Euler_280k_sc_gauges/_gauges/gauge650.25.txt")
plt.plot(v*(tt-0.26)+shift,p,"r")

tt, p = get_pressure("./_Euler_280k_sc_gauges/_gauges/gauge650.5.txt")
plt.plot(v*(tt-0.64)+shift,p,"r")

plt.xlim(-4,4)
from scipy.io import loadmat

data = loadmat("/Users/ketch/Dropbox/Ketcheson/Compressible_gas/travel_data6.mat")
pressure = 1 + v*data["u2"]#*1.05
plt.plot(data["TT"]-6.7,pressure,"--k",alpha=0.5,label="2nd-order approximation");

pressure = 1 + v*data["u4"]#*1.05
plt.plot(data["TT"]-6.7,pressure,"--k",label="4th-order approximation");

plt.xlabel(r"$\xi$")
plt.ylabel(r"Pressure");
plt.legend(fontsize=11)
plt.savefig("solitary_wave_comparison.pdf",bbox_inches="tight")

# Shock-tube entropy experiments

To generate the data for these figures, run the following command:

`python3 entropy_front_experiments.py pl=2.0 mx=40000 L=50 scalefac=2.0 tfinal=15.0`

but with `mx` taking each of the values `{10000, 20000, 40000, 80000}` and `pl` taking each of the values `{1.25, 1.5, 1.75, 2.0, 2.25, 2.5}`.

In [ ]:
import entropy_front_experiments

def get_entropy(sol):
    rho = sol.state.q[0,:]
    rhou = sol.state.q[1,:]
    E = sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    dx = sol.grid.x.delta
    
    return dx*np.sum(rho*np.log(P/rho**gamma))

def local_entropy(sol):
    rho = sol.state.q[0,:]
    rhou = sol.state.q[1,:]
    E = sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    
    return rho*np.log(P/rho**gamma)

def local_entropy_flux(sol):
    S = local_entropy(sol)
    u = sol.state.q[1,:]/sol.state.q[0,:]
    return u*S
    
def get_pressure(i,claw):
    sol = claw.frames[i]
    xc = sol.state.grid.x.centers
    rho = sol.state.q[0,:]
    rhou = sol.state.q[1,:]
    E = sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    t = sol.t
    return xc, P

def cmax(rho,pr):
    integrand = lambda x: 1./np.sqrt(rho(x))
    return np.sqrt(gamma*pr)*quad(integrand,0,1)[0]

rhofun = lambda x: 1 + 0.8*np.cos(2*np.pi*x)
pr = 1.
c_max = cmax(rhofun,pr)

prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

font = {'size'   : 10}
matplotlib.rc('font', **font)

In [ ]:
mx = 20000
L = 50
tfinal = 15.
t = tfinal
fig, ax = plt.subplots(3,2,figsize=(10,8), sharex=True)
fig.subplots_adjust(hspace=0)
props = dict(facecolor='lightgrey', alpha=0.2)
xtext = 5

pl = 1.25
claw = entropy_front_experiments.setup(pl=pl,mx=mx,tfinal=tfinal,L=L,solver_type="classic",nout=5)
claw.verbosity=0
claw.run()
xc, P = get_pressure(0,claw)
ax[0][0].plot(xc,P,"k",alpha=0.5)
xc, P = get_pressure(-1,claw)
ax[0][0].plot(xc,P, color=colors[0])
pmax = np.max(P)
ax[0][0].plot([c_max*t,c_max*t],[1,pmax],"--k",alpha=0.5);
initial_entropy = get_entropy(claw.frames[0])
final_entropy = get_entropy(claw.frames[-1])
entropy_change = (final_entropy-initial_entropy)
lep = claw.problem_data["max_local_ent_prod"]
ax[0][0].text(xtext,1.2,r'$p_\ell=%3.2f$' % (pl),bbox=props)

#ax[0][0].set_title(str(lep))

pl = 1.5
claw = entropy_front_experiments.setup(pl=pl,mx=mx,tfinal=tfinal,L=L,solver_type="classic",nout=5)
claw.verbosity=0
claw.run()
xc, P = get_pressure(-1,claw)
ax[0][1].plot(xc,P, color=colors[1])
pmax = np.max(P)
ax[0][1].plot([c_max*t,c_max*t],[1,pmax],"--k",alpha=0.5);
initial_entropy = get_entropy(claw.frames[0])
final_entropy = get_entropy(claw.frames[-1])
lep = claw.problem_data["max_local_ent_prod"]
ax[0][1].text(xtext,1.4,r'$p_\ell=%3.1f$' % (pl),bbox=props)

pl = 1.75
claw = entropy_front_experiments.setup(pl=pl,mx=mx,tfinal=tfinal,L=L,solver_type="classic",nout=5)
claw.verbosity=0
claw.run()
xc, P = get_pressure(-1,claw)
ax[1][0].plot(xc,P, color=colors[2])
pmax = np.max(P)
ax[1][0].plot([c_max*t,c_max*t],[1,pmax],"--k",alpha=0.5);
initial_entropy = get_entropy(claw.frames[0])
final_entropy = get_entropy(claw.frames[-1])
lep = claw.problem_data["max_local_ent_prod"]
ax[1][0].text(xtext,1.6,r'$p_\ell=%3.2f$' % (pl),bbox=props)

pl = 2.0
claw = entropy_front_experiments.setup(pl=pl,mx=mx,tfinal=tfinal,L=L,solver_type="classic",nout=5)
claw.verbosity=0
claw.run()
xc, P = get_pressure(-1,claw)
ax[1][1].plot(xc,P, color=colors[3])
pmax = np.max(P)
ax[1][1].plot([c_max*t,c_max*t],[1,pmax],"--k",alpha=0.5);
initial_entropy = get_entropy(claw.frames[0])
final_entropy = get_entropy(claw.frames[-1])
lep = claw.problem_data["max_local_ent_prod"]
ax[1][1].text(xtext,1.8,r'$p_\ell=%3.1f$' % (pl),bbox=props)

pl = 2.25
claw = entropy_front_experiments.setup(pl=pl,mx=mx,tfinal=tfinal,L=L,solver_type="classic",nout=5)
claw.verbosity=0
claw.run()
xc, P = get_pressure(-1,claw)
ax[2][0].plot(xc,P, color=colors[4])
pmax = np.max(P)
ax[2][0].plot([c_max*t,c_max*t],[1,pmax],"--k",alpha=0.5);
initial_entropy = get_entropy(claw.frames[0])
final_entropy = get_entropy(claw.frames[-1])
lep = claw.problem_data["max_local_ent_prod"]
ax[2][0].text(xtext,2.1,r'$p_\ell=%3.2f$' % (pl),bbox=props)

pl = 2.5
claw = entropy_front_experiments.setup(pl=pl,mx=mx,tfinal=tfinal,L=L,solver_type="classic",nout=5)
claw.verbosity=0
claw.run()
xc, P = get_pressure(-1,claw)
ax[2][1].plot(xc,P, color=colors[5])
pmax = np.max(P)
ax[2][1].plot([c_max*t,c_max*t],[1,pmax],"--k",alpha=0.5);
initial_entropy = get_entropy(claw.frames[0])
final_entropy = get_entropy(claw.frames[-1])
lep = claw.problem_data["max_local_ent_prod"]
ax[2][1].text(xtext,2.3,r'$p_\ell=%3.1f$' % (pl),bbox=props)

for axis in ax.reshape(-1):
    axis.set_xlim(-40,30)
ax[0][0].legend(["Pressure at t=0","Pressure at t=15",r"$x=c_\text{max} t$"])
plt.savefig("shocktubes.pdf");

In [ ]:
mx_vals = [10000, 20000, 40000, 80000]
pl_vals = [1.25, 1.5, 1.75, 2.0, 2.25, 2.5]
lepmax = np.zeros((len(pl_vals),len(mx_vals)))

for i, mx in enumerate(mx_vals):
    for j, pl in enumerate(pl_vals):
        path = "./entropy_front/"+str(pl)+"_"+str(mx)
        #print(path)
        try:
            sol = pyclaw.Solution(5,path=path,read_aux=False)
        except FileNotFoundError:
            sol = pyclaw.Solution(5,file_format='petsc',file_prefix='claw',path=path,read_aux=False)
        lepmax[j,i] = sol.state.problem_data["max_local_ent_prod"]

for j in range(len(pl_vals)):
    plt.plot(np.array(mx_vals),lepmax[j,:],label=r"$p_\ell = $"+str(pl_vals[j]))

plt.xlabel(r"$m_x$");
plt.ylabel(r"$\max_{n,j} \ \ \eta^n_j$")
plt.legend();
plt.savefig("max_loc_ent_prod.pdf")

# Random medium entropy experiment

The Clawpack data for the following plot is generated by running

`python3 Eulerian_IC.py xright=256 mx=102400 tfinal=200 nout=100 use_petsc=True entropy_field="rho_random_84297_L256_dx200_smooth320k.txt" outdir='./_random_b_L256_dx200_smooth320k' solver_type=sharpclaw`

`mpirun -np 6 python3 Eulerian_IC.py xright=256 mx=102400 tfinal=200 nout=100 use_petsc=True entropy_field="rho_random_4569_L256_dx200.txt" outdir='./_random_L256_dx200' solver_type=sharpclaw`

In [ ]:
def get_pressure(sol):
    xc = sol.state.grid.x.centers
    rho = sol.state.q[0,:]
    rhou = sol.state.q[1,:]
    E = sol.state.q[2,:]
    P = (gamma-1)*(E-0.5*rhou**2/rho)
    t = sol.t
    return xc, P
    
fig = plt.figure(figsize=(10,8))
#fig, ax = plt.subplots(1,2,figsize=(10,4),sharey=True)
fig.subplots_adjust(hspace=0)

path = "./_random_b_L256_dx200_smooth320k/"
final_sol = pyclaw.Solution(100,file_format='petsc',file_prefix='claw',path=path,read_aux=False)
xc, pressure = get_pressure(final_sol)
plt.subplot(221)
plt.plot(xc,pressure)
plt.xlim(200,256)
plt.xlabel("x");
plt.ylabel("Pressure")

path = "./_random_L256_dx200"
final_sol = pyclaw.Solution(100,file_format='petsc',file_prefix='claw',path=path,read_aux=False)
xc, pressure = get_pressure(final_sol)
plt.subplot(222)
plt.plot(xc,pressure)
plt.xlim(200,256)
plt.xlabel("x");

plt.subplot(313)
rho = final_sol.state.q[0,:]
plt.plot(xc,rho)
plt.xlim(50,100);
plt.xlabel("x");
plt.ylabel("Density");
plt.savefig("random.pdf");